In [1]:
import time
import pandas as pd
import numpy as np

import geopandas as gpd
from shapely.geometry import Point, Polygon

In [2]:
blocks = gpd.read_file('data/Census_Blocks__2010.geojson')
blocks.crs = {'init': 'epsg:4326'}
blocks = blocks[['GEOID', 'BLOCK', 'BLKGRP', 'P0010001', 'SqMiles', 'geometry']]
blks = blocks.set_index('GEOID')

In [3]:
## Impervious surfaces data can be found on opendata.dc.gov
## Shape file loads more quickly than geojson in this case
print ('Started pulling data at '+time.strftime("%a, %d %b %Y %H:%M:%S +0000", time.localtime())) 
dcgeo = gpd.read_file('data/Impervious_Surface_2015.shp')
dcgeo = dcgeo[dcgeo.geometry.isnull()==False]
print ('Finished pulling data at '+time.strftime("%a, %d %b %Y %H:%M:%S +0000", time.localtime())) 
dcgeo = dcgeo[['Shape_Area', 'geometry']]
dcgeo.columns = ['impervious_area', 'geometry']

Started pulling data at Tue, 10 Jul 2018 18:07:16 +0000
Finished pulling data at Tue, 10 Jul 2018 18:07:51 +0000


In [4]:
## Spatial join 311 data points to Census block polygons
print ('Started spatial join at '+time.strftime("%a, %d %b %Y %H:%M:%S +0000", time.localtime())) 
geo_df = gpd.sjoin(blocks, dcgeo, how='left', op='intersects')
print ('Finished spatial join at '+time.strftime("%a, %d %b %Y %H:%M:%S +0000", time.localtime())) 
geo_df.head()

Started spatial join at Tue, 10 Jul 2018 18:07:51 +0000
Finished spatial join at Tue, 10 Jul 2018 18:08:23 +0000


,GEOID,BLOCK,BLKGRP,P0010001,SqMiles,geometry,index_right,impervious_area
0,110010003003006,3006,0003003,251,0.038968,"POLYGON ((-77.07863030944546 38.9161507873491,...",185921.0,258.786174
0,110010003003006,3006,0003003,251,0.038968,"POLYGON ((-77.07863030944546 38.9161507873491,...",80608.0,103.488579
0,110010003003006,3006,0003003,251,0.038968,"POLYGON ((-77.07863030944546 38.9161507873491,...",80595.0,107.885565
0,110010003003006,3006,0003003,251,0.038968,"POLYGON ((-77.07863030944546 38.9161507873491,...",80562.0,106.859075
0,110010003003006,3006,0003003,251,0.038968,"POLYGON ((-77.07863030944546 38.9161507873491,...",80549.0,115.629443


In [5]:
cols = dcgeo.drop(['geometry'], axis=1).columns
data = geo_df.groupby('GEOID')[cols].sum()
blks = blks.merge(data, how='left', left_index=True, right_index=True)

In [6]:
blks.index.value_counts().head()

110010095053005    1
110010062021158    1
110010077091004    1
110010047021002    1
110010077072025    1
Name: GEOID, dtype: int64

In [7]:
blks = blks.drop(['BLOCK', 'BLKGRP', 'P0010001', 'SqMiles', 'geometry'], axis=1)

In [8]:
blks.to_csv('data/impervious_surfaces.csv.gz', compression = 'gzip')